In [1]:
import os
import torch
import torch.nn as nn
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

#from keras.models import Sequential, Model
#from keras.layers import Dense, Input
import datatable as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import joblib
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
import os
from keras.models import load_model
import joblib
from sklearn.neural_network import MLPClassifier
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
PATH='/home/jm/SNUH/methylation/v2/compare/'


os.environ["CUDA_VISIBLE_DEVICES"]="1" # 0, 1, 2, 3 중 하나
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4 # GPU 점유 비율
session = InteractiveSession(config=config)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

                        

2023-06-07 11:30:38.420525: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


cuda


2023-06-07 11:31:25.524945: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19470 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:41:00.0, compute capability: 8.6


# Data Loading

In [2]:
data=pd.read_csv('/workspace/jaeminjeon_950515/SNUH/school/bioinfo1/35L33G.csv')
data['labels']='binding_Motif'
data1=pd.read_csv('/workspace/jaeminjeon_950515/SNUH/school/bioinfo1/control.csv')

data1.columns=['results','labels']
data1['labels']='control'
data=pd.concat([data,data1])
df=data.drop_duplicates('results')
df=df.reset_index(drop=True)

In [3]:
df=df.copy()

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from transformers import AutoTokenizer, AutoModel
model_name = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
#model_name = "BMILab/TCR-BERT-MLM"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(4, 32, kernel_size=3)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * (sequence_length - 4), 128)  # Adjust input size of fc1 layer
        self.fc2 = nn.Linear(128, 2)  # Assuming 2 classes for prediction

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# encoding

In [8]:
from sklearn.model_selection import train_test_split

# Load the data
data = df.loc[:,'results']
train_sequences = [''.join(list(word)) for word in data]
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(list(df['labels']))

In [111]:
df.loc[:,'results']

0        GTGGGTAAGAGCACCCGA
1        CTTCTGGAGTGTCTGAAG
2        CATAATTTGTGGTAGTGG
3        CTCTTCTGGAGTGTCTGA
4        CTTCTGGTGTGTCTGAAG
                ...        
24563    AAAGATGAGGTCTGTTTG
24564    ACCCAGAAGACTGTGGAT
24565    ATTCTCACGGAGGAAGGA
24566    AGGCACACGCGGCACACA
24567    GCACACGCGGCACACACG
Name: results, Length: 24568, dtype: object

In [9]:
encoded_matrices = tokenizer(train_sequences, truncation=True, padding=True, return_tensors='pt')
input_ids = encoded_matrices['input_ids'].to(device)
attention_mask = encoded_matrices['attention_mask'].to(device)
#label_batch1 = torch.tensor(train_labels).to(device)

In [10]:
bert_model = AutoModel.from_pretrained(model_name).to(device)
a=list()
for i in range(0,input_ids.shape[0]):
    
    outputs = bert_model(input_ids[i:i+1],attention_mask[i:i+1])
    a.append(outputs.last_hidden_state[:, 0, :].cpu().detach().numpy())

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
Some weights of the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref were not used when initializing EsmModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing EsmModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceCla

In [113]:
import numpy as np

# Assuming 'a' is the NumPy array you want to save
a=np.array(a).reshape(24568, 6, 128)
np.save('/workspace//jaeminjeon_950515/SNUH/school/bioinfo1/BERT_encoded.npy', a)

In [14]:
a=np.array(a).reshape(24568, 10, 128)

In [13]:
np.array(a).shape

(24568, 1, 1280)

In [81]:
b = np.array(a).reshape(24568, 35, 768)

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

# Define the CNN model
class MLPClassifier(nn.Module):
        def __init__(self, hidden_dim=128, n_classes=2,seed=1234):
            super().__init__()
            torch.manual_seed(seed)
            self.hidden_dim = hidden_dim
            self.n_classes = n_classes
    
            self.fc1 = nn.Linear(1280, self.hidden_dim)
            self.fc2 = nn.Linear(self.hidden_dim, self.n_classes)
    
        def forward(self, x):
            x = self.fc1(x)
            x = nn.functional.relu(x)
            x = self.fc2(x)
            return x
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        #self.conv1 = nn.Conv1d(6, 32, kernel_size=3)
        #self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        #self.fc1 = nn.Linear(7936, 128)  # Adjust input size of fc1 layer
        #self.fc2 = nn.Linear(128, 2)  # Assuming 2 classes for prediction
        self.conv1 = nn.Conv1d(10, 32, kernel_size=3)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(48896, 128)  # Adjust input size of fc1 layer
        self.fc2 = nn.Linear(128, 2)  # Assuming 2 classes for prediction
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


# Convert the list of matrices to a numpy array
encoded_matrices = np.array(encoded_matrices)

labels = np.array(df['labels'].tolist())

# Encode labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and validation sets
#a=np.array(a).reshape(24568, 6, 128)
a=np.array(a).reshape(24568, 10, 128)
#a=np.array(a).reshape(24568, 35, 768)
data_train, data_val, labels_train, labels_val = train_test_split(a, labels, test_size=0.2, random_state=42, stratify=labels)


In [62]:
a.shape

(24568, 1280, 1)

# MOdel

In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
class MLPClassifier(nn.Module):
        #def __init__(self, hidden_dim=128, n_classes=2,seed=1234):
        def __init__(self):
            super(MLPClassifier, self).__init__()
            #torch.manual_seed(seed)
            self.hidden_dim = 640
            self.n_classes =2
    
            self.fc1 = nn.Linear(1280, self.hidden_dim)
            self.fc2 = nn.Linear(self.hidden_dim, self.n_classes)
    
        def forward(self, x):
            x = self.fc1(x)
            x = nn.functional.relu(x)
            x = self.fc2(x)
            return x
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(10, 32, kernel_size=3)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(7936, 16)  # Adjust input size of fc1 layer
        self.fc2 = nn.Linear(16, 2)  # Assuming 2 classes for prediction
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


# Convert the list of matrices to a numpy array
encoded_matrices = np.array(encoded_matrices)

labels = np.array(df['labels'].tolist())

# Encode labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and validation sets
#a=np.array(a).reshape(24568, 6, 128)
a=np.array(a).reshape(24568, 1280)
#a=np.array(a).reshape(24568, 35, 768)
data_train, data_val, labels_train, labels_val = train_test_split(a, labels, test_size=0.2, random_state=42, stratify=labels)

# Create custom datasets
train_dataset = CustomDataset(data_train, labels_train)
val_dataset = CustomDataset(data_val, labels_val)

# Create data loaders
batch_size = 300
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Instantiate the CNN model
#sequence_length = encoded_matrices.shape[1]  # Get sequence length
sequence_length=12
#model = CNNModel()
model=MLPClassifier()
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.8)

# Training loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
for epoch in range(num_epochs):
    print(epoch)
    model.train()
    for data, labels in train_loader:
        #print(data.shape)
        #data = data.float().permute(0, 1, 2).to(device)  # Permute dimensions for proper input shape
        data = data.float().to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        print(loss)
        loss.backward()
        optimizer.step()

0
tensor(0.6656, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6644.8799, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(720.4700, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(7454.0439, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4465.1167, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(28.2486, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(122.4262, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(281.0775, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(309.9739, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(277.2490, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(266.6916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(254.1498, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(149.9673, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(69.0567, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(58.7893, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(27.5291, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.

In [74]:
pd.DataFrame(labels_train).value_counts()

0    18279
1     1375
dtype: int64

In [80]:
model.eval()
predictions = []
with torch.no_grad():
    for data, _ in val_loader:
        #data = data.float().permute(0, 1, 2).to(device)
        data = data.float().to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())

In [81]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, \
    classification_report, precision_recall_fscore_support, roc_auc_score
from sklearn import metrics

In [82]:
print(metrics.classification_report(label_encoder.inverse_transform(predictions),
                                    label_encoder.inverse_transform(labels_val), digits=4))

               precision    recall  f1-score   support

binding_Motif     1.0000    0.9300    0.9637      4914
      control     0.0000    0.0000    0.0000         0

     accuracy                         0.9300      4914
    macro avg     0.5000    0.4650    0.4819      4914
 weighted avg     1.0000    0.9300    0.9637      4914



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from torch.nn.functional import softmax
model.eval()
predictions = []
predictions_prob = []
with torch.no_grad():
    for data, _ in val_loader:
        data = data.float().permute(0, 1, 2).to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        prob=softmax(outputs.data, dim=1)
        predictions_prob.extend(prob.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())

In [ ]:
pd.DataFrame(np.array(predictions_prob)[:,1]).to_csv('/workspace/jaeminjeon_950515/SNUH/school/bioinfo1/DNA_BERT_prob_hidden1-1.csv')

In [ ]:
pd.DataFrame(np.array(predictions_prob)[:,0]).to_csv('/workspace/jaeminjeon_950515/SNUH/school/bioinfo1/DNA_BERT_prob.csv')

In [ ]:
a=np.array(a).reshape(24568, 6, 128)
#a=np.array(a).reshape(24568, 35, 768)
data_train, data_val, labels_train, labels_val = train_test_split(a, labels, test_size=0.2, random_state=42, stratify=labels)
